In [ ]:
import pandas as pd , numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [ ]:
train = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')
test = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv')
subm = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv')

In [ ]:
# Look at The data

train.head()

In [ ]:
train['comment_text'][0]

In [ ]:
train['comment_text'][2]

In [ ]:
lens = train.comment_text.str.len()
lens.mean(),lens.std(),lens.max() , lens.mean() , lens.median()

In [ ]:
lens.hist();

In [ ]:
label_cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
train['none'] = 1 - train[label_cols].max(axis = 1)
train.describe()

In [ ]:
len(train),len(test)

In [ ]:
COMMENT = 'comment_text'
train[COMMENT].fillna('unknown',inplace = True)
test[COMMENT].fillna('unknown',inplace = True)

In [ ]:
# Tokenize
import re,string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s):return re_tok.sub(r'\1',s).split()

In [ ]:
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2),tokenizer=tokenize,
                     min_df=3,max_df=0.9,strip_accents='unicode',use_idf = 1,
                     smooth_idf=1,sublinear_tf=1)
trn_term_doc = vec.fit_transform(train[COMMENT])
test_term_doc = vec.transform(test[COMMENT])

In [ ]:
trn_term_doc , test_term_doc

In [ ]:
def pr(y_i , y):
    p = x[y==y_i].sum()
    return ( p + 1) / ((y == y_i).sum()+1)

In [ ]:
x = trn_term_doc
test_x = test_term_doc

In [ ]:
def get_mdl(y):
    y  = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m  = LogisticRegression(C = 4 , dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb,y),r


In [ ]:
preds = np.zeros((len(test),len(label_cols)))

for i , j in enumerate(label_cols):
    
    print('fit' , j)
    m , r = get_mdl(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]
    

In [ ]:
submid = pd.DataFrame({'id': subm['id']})
submission = pd.concat([submid , pd.DataFrame(preds , columns = label_cols)],axis = 1)
submission.to_csv('submission.csv',index = False)